In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from pprint import pprint
from IPython.display import display

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt

In [3]:
from model_logistic_regression import y, X, idx, num_test
from model_logistic_regression import scaler, model
from turnpt_analysis import get_daily_TXF_OHLC

transformed OHLCV at frequency 1D
reading cache of TXF in 1D OHLCV


/home/cyyen/miniconda3/envs/stock_path_tracker/lib/python3.8/site-packages/pandas/core/groupby/groupby.py:980: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  result = concat(values, axis=self.axis, keys=keys)


Accuracy Score: 0.5983086680761099
Confusion Matrix:


,Negative,Positve
False,262,189
True,1,21


Area Under Curve: 0.7677383592017739
Recall score: 0.9545454545454546


In [4]:
X_scaled = scaler.transform(X)
X_, y_, idx_ = X_scaled[-num_test:], y[-num_test:], idx[-num_test:]

In [5]:
signal = model.predict(X_)
signal

array([0., 0., 1., 1., 1., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 1., 1., 0., 1.,
       1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 1., 1., 1., 0., 0., 0.,
       0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1.,
       1., 1., 1., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 1., 0.,
       1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1.,
       0., 0., 0., 1., 1., 0., 0., 0., 1., 0., 1., 1., 1., 1., 1., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 0., 0., 0.,
       0., 0., 0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 1., 0., 0., 0.

In [16]:
def get_position(df, holding_days=5):
    df['exit'] = 0
    df['exit_at'] = np.nan
    df['exit_at'] = np.where((df.enter == 1) & (df.p0 > df.p5), 'p5', df.exit_at)# NOTE: order is important
    df['exit_at'] = np.where((df.enter == 1) & (df.p0 > df.p4), 'p4', df.exit_at)
    df['exit_at'] = np.where((df.enter == 1) & (df.p0 > df.p3), 'p3', df.exit_at)
    df['exit_at'] = np.where((df.enter == 1) & (df.p0 > df.p2), 'p2', df.exit_at)
    df['exit_at'] = np.where((df.enter == 1) & (df.p0 > df.p1), 'p1', 'p_holding')
    assert df.exit_at.isna().sum() == 0
    
    df['prc_exit'] = np.nan
    df['prc_exit'] = np.where(df.exit_at == 'p5', df.p5, df.prc_exit)
    df['prc_exit'] = np.where(df.exit_at == 'p4', df.p4, df.prc_exit)
    df['prc_exit'] = np.where(df.exit_at == 'p3', df.p3, df.prc_exit)
    df['prc_exit'] = np.where(df.exit_at == 'p2', df.p2, df.prc_exit)
    df['prc_exit'] = np.where(df.exit_at == 'p1', df.p1, df.prc_exit)
    df['prc_exit'] = np.where(df.exit_at == 'p_holding', df.p0.shift(-holding_days), df.prc_exit)
    
    days_enter = df.query('enter == 1').index
    nrows = df.shape[0]
    for idx_row in range(nrows):
        dt, row = df.index[idx_row], df.iloc[idx_row]
        if dt in days_enter:
            if row.exit_at == 'p_holding':
                idx_exit = idx_row+holding_days
            elif row.exit_at == 'p1':
                idx_exit = idx_row+1
            elif row.exit_at == 'p2':
                idx_exit = idx_row+2
            elif row.exit_at == 'p3':
                idx_exit = idx_row+3
            elif row.exit_at == 'p4':
                idx_exit = idx_row+4
                
            elif row.exit_at == 'p5':
                idx_exit = idx_row+5
    
            else:
                raise ValueError(f'There is no such exit_at code: {row.exit_at}')
                
            
            if idx_exit <= nrows-1:
                dt_exit = df.index[idx_exit]
                df.loc[dt_exit, 'exit'] += 1

    buy = df.enter.cumsum()
    sell = df.exit.cumsum()
    df['position'] = buy - sell
    return df


def get_df_action(dt_includes, signal, holding_days):
    df = (
        get_daily_TXF_OHLC().
        set_index('tx_datetime').loc[:, ['close']].
        query('index in @idx_').
        assign(signal = signal, 
               signal_true = y_).
        assign(p0 = lambda x: x.close,
               p_1 = lambda x: x.close.shift(1), p_2 = lambda x: x.close.shift(2), p_3 = lambda x: x.close.shift(3), p_4 = lambda x: x.close.shift(4), p_5 = lambda x: x.close.shift(5),
               p1 = lambda x: x.close.shift(-1), p2 = lambda x: x.close.shift(-2), p3 = lambda x: x.close.shift(-3), p4 = lambda x: x.close.shift(-4), p5 = lambda x: x.close.shift(-5),).
        dropna().
        assign(enter = lambda x: np.where(x[['p0', 'p_1', 'p_2', 'p_3', 'p_4', 'p_5']].idxmin(axis=1) == 'p0', x.signal, 0)).
        pipe(get_position, holding_days = holding_days).
        assign(action = lambda x: x.position.diff().fillna(x.iloc[0].position))
    )
    return df

In [17]:
if __name__ == '__main__':
    holding_days = 5
    df_action = get_df_action(idx_, signal, holding_days)
    display(df_action.head(15).loc[:, ['signal_true', 'signal', 'enter', 'exit_at', 'position', 'action']])
    
    # correlation
    df_signal_confirmed = (
        df_action.
        assign(signal_enter = lambda x: x.enter == 1,
               signal_t1 = lambda x: ((x.enter == 1) & (~x.exit_at.isin(['p1']))).astype(int),
               signal_t2 = lambda x: ((x.enter == 1) & (~x.exit_at.isin(['p1', 'p2']))).astype(int),
               signal_t3 = lambda x: ((x.enter == 1) & (~x.exit_at.isin(['p1', 'p2', 'p3']))).astype(int),
               signal_t4 = lambda x: ((x.enter == 1) & (~x.exit_at.isin(['p1', 'p2', 'p3', 'p4']))).astype(int),
               signal_t5 = lambda x: ((x.enter == 1) & (~x.exit_at.isin(['p1', 'p2', 'p3', 'p4', 'p5']))).astype(int)).
        loc[:, ['signal_true', 'signal', 'signal_enter', 'signal_t1', 'signal_t2', 'signal_t3', 'signal_t4', 'signal_t5']]
    )
    display(round(df_signal_confirmed.corr(), 2))


transformed OHLCV at frequency 1D
reading cache of TXF in 1D OHLCV


,signal_true,signal,enter,exit_at,position,action
tx_datetime,,,,,,
2017-03-03,0.0,1.0,1.0,p_holding,1.0,1.0
2017-03-06,0.0,0.0,0.0,p_holding,1.0,0.0
2017-03-07,0.0,0.0,0.0,p_holding,1.0,0.0
2017-03-08,0.0,0.0,0.0,p_holding,1.0,0.0
2017-03-09,0.0,1.0,0.0,p_holding,1.0,0.0
2017-03-10,1.0,1.0,1.0,p_holding,1.0,0.0
2017-03-13,0.0,0.0,0.0,p_holding,1.0,0.0
2017-03-14,0.0,0.0,0.0,p_holding,1.0,0.0
2017-03-15,0.0,0.0,0.0,p_holding,1.0,0.0


,signal_true,signal,signal_enter,signal_t1,signal_t2,signal_t3,signal_t4,signal_t5
signal_true,1.00,0.28,0.42,0.61,0.61,0.61,0.61,0.61
signal,0.28,1.00,0.62,0.44,0.44,0.44,0.44,0.44
signal_enter,0.42,0.62,1.00,0.71,0.71,0.71,0.71,0.71
signal_t1,0.61,0.44,0.71,1.00,1.00,1.00,1.00,1.00
signal_t2,0.61,0.44,0.71,1.00,1.00,1.00,1.00,1.00
signal_t3,0.61,0.44,0.71,1.00,1.00,1.00,1.00,1.00
signal_t4,0.61,0.44,0.71,1.00,1.00,1.00,1.00,1.00
signal_t5,0.61,0.44,0.71,1.00,1.00,1.00,1.00,1.00
